In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Define directory containing all images
data_dir = '/content/drive/MyDrive/Monkeypox Skin Image Dataset'

In [5]:
img_size = (150, 150)
batch_size = 32

# Data Augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [6]:
# Load Data
generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',  # because there are only two classes
    shuffle=True
)

Found 572 images belonging to 2 classes.


In [7]:
# Define class labels
class_labels = list(generator.class_indices.keys())

# Load pre-trained VGG16 model
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze layers in VGG16
for layer in base_model.layers:
    layer.trainable = False

# Model Definition


58889256/58889256 [==============================] - 0s 0us/step


In [8]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),  # Replace Flatten with GlobalAveragePooling2D
    tf.keras.layers.Dense(512, activation='relu', input_shape=(base_model.output_shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification
])

In [9]:

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=1000,
    decay_rate=0.9
)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='binary_crossentropy',
              metrics=['accuracy'])



In [10]:
history = model.fit(
    generator,
    steps_per_epoch=generator.samples // batch_size,
    epochs=10,
)

Epoch 1/10
17/17 [==============================] - 138s 8s/step - loss: 0.5320 - accuracy: 0.7315
Epoch 2/10
17/17 [==============================] - 130s 8s/step - loss: 0.3763 - accuracy: 0.8259
Epoch 3/10
17/17 [==============================] - 127s 8s/step - loss: 0.3280 - accuracy: 0.8611
Epoch 4/10
17/17 [==============================] - 134s 8s/step - loss: 0.2896 - accuracy: 0.8889
Epoch 5/10
17/17 [==============================] - 130s 8s/step - loss: 0.2703 - accuracy: 0.8981
Epoch 6/10
17/17 [==============================] - 130s 8s/step - loss: 0.2539 - accuracy: 0.9074
Epoch 7/10
17/17 [==============================] - 130s 8s/step - loss: 0.2336 - accuracy: 0.9000
Epoch 8/10
17/17 [==============================] - 133s 8s/step - loss: 0.2308 - accuracy: 0.9148
Epoch 9/10
17/17 [==============================] - 128s 8s/step - loss: 0.2125 - accuracy: 0.9259
Epoch 10/10
17/17 [==============================] - 128s 7s/step - loss: 0.1979 - accuracy: 0.9278


In [12]:
# Save the model
model.save("/content/monkeypox_disease_model.h5")
# Prediction

In [16]:
from tensorflow.keras.preprocessing import image

def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

def predict_(model, image_path):
    processed_image = preprocess_image(image_path)
    prediction = model.predict(processed_image)
    if prediction > 0.5:
        return class_labels[0], prediction[0][0]
    else:
        return class_labels[1], 1 - prediction[0][0]

image_path = '/content/drive/MyDrive/Monkeypox Skin Image Dataset/Normal/normal100.png'  # Replace with the actual path to your image
predicted, confidence = predict_(model, image_path)
print("Predicted Class:", predicted)
print("Confidence:", confidence)

1/1 [==============================] - 0s 219ms/step
Predicted Class: Normal
Confidence: 0.5705140829086304
